# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
Vacation_data= "../VacationPY/output_data/cities.csv"
vacationData=pd.read_csv(Vacation_data)

vacationData.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [ ]:
#Use the Lat and Lng as locations and Humidity as the weight.

locations = vacationData[["Lat", "Lng"]].astype(float)

#humidity
humidity = vacationData["Humidity"].astype(float)

#dimensions for graph
figure_layout = {
     'width' : '1000px',
     'height':'400px',
     'border':'1px solid black',
     'padding':'1px',
     'margin':'0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=100,
                                   point_radius=5)
fig.add_layer(heatmap_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
temperatureCriteria = vacationData['Max Temp'] < 75
temperatureCriteria = vacationData['Max Temp'] > 55
windSpeedCriteria = vacationData['Wind Speed'] < 10
cloudinessCriteria = vacationData['Cloudiness'] == 0
Wholecriteria = temperatureCriteria & windSpeedCriteria & windSpeedCriteria 

# Use boolean indexing to filter the weather_df dataframe
weather_df = vacationData[Wholecriteria]
weather_df = weather_df.dropna()
weather_df = weather_df.reset_index()
weather_df.head(10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#hotel_df = pd.DataFrame(weather_df, columns=['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Hotel Name'])
#hotel_df

hotel_df = pd.DataFrame(data=vacationData)
hotel_df

In [ ]:
#Add a "Hotel Name" column to the DataFrame.

hotel_df.loc[:,'Hotel Name'] = np.nan
hotel_df

In [ ]:
target_radius = 5000
target_type = "hotel"
target_search = "hotel"
hotel_df.loc[:,"Coordinates"] = hotel_df.loc[:,"Lat"].astype(str) +"," + hotel_df.loc[:,"Lng"].astype(str)

hotel_list = []

for location in hotel_df["Coordinates"]:
    params = {
        "location": location,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    gmaps_json = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    
    try:
        hotel_list.append(gmaps_json["results"][0]["name"])
        
    except:
        hotel_list.append("N/A")
        print(f"No hotels within 5 miles of {location}.")

hotel_df["Hotel Name"] = hotel_list
hotel_df.loc[:, ~hotel_df.columns.str.contains('^Unnamed')]


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

In [ ]:
"/outputdata/Heatmap.png"